# A World with Things to Do

So far, our 2D world has been empty—just agents moving and talking. But a real virtual world needs **objects** and **activities**. Agents should be able to discover items, interact with them, and discuss what they find.

This post adds:
- **Objects**: Items that exist in the world at specific locations
- **Activities**: Actions agents can perform on objects
- **Discovery**: Agents can find objects as they explore
- **Interaction**: Agents can use objects, which creates new conversation topics

## Object Types

We'll implement several object types:
- **Tools**: Items that can be used (e.g., "hammer", "key")
- **Landmarks**: Interesting locations (e.g., "fountain", "tree")
- **Items**: Collectible objects (e.g., "coin", "book")


## Setup and Imports


In [1]:
import os
import re
from dataclasses import dataclass, field
from typing import List, Optional, Dict
from enum import Enum
from dotenv import load_dotenv
from openai import OpenAI
import random

load_dotenv("../../.env")
client = OpenAI()

GRID_SIZE = 20


## Object and Activity Definitions


In [2]:
class ObjectType(Enum):
    TOOL = "tool"
    LANDMARK = "landmark"
    ITEM = "item"

@dataclass
class WorldObject:
    """An object that exists in the world."""
    name: str
    x: int
    y: int
    obj_type: ObjectType
    description: str
    interactable: bool = True
    
    def get_position(self):
        return (self.x, self.y)
    
    def distance_to(self, x: int, y: int) -> float:
        """Calculate distance to a point."""
        return ((self.x - x)**2 + (self.y - y)**2)**0.5

class Activity:
    """An activity that can be performed on an object."""
    
    @staticmethod
    def get_activity_description(obj: WorldObject, agent_name: str) -> str:
        """Generate a description of interacting with an object."""
        if obj.obj_type == ObjectType.TOOL:
            return f"{agent_name} uses the {obj.name}."
        elif obj.obj_type == ObjectType.LANDMARK:
            return f"{agent_name} examines the {obj.name}."
        elif obj.obj_type == ObjectType.ITEM:
            return f"{agent_name} picks up the {obj.name}."
        return f"{agent_name} interacts with the {obj.name}."


## Enhanced Agent Class

Agents need to be aware of objects in the world.


In [3]:
@dataclass
class Agent:
    name: str
    x: int
    y: int
    color: str
    history: list = field(default_factory=list)
    discovered_objects: list = field(default_factory=list)
    
    def move(self, dx, dy):
        self.x = max(0, min(GRID_SIZE-1, self.x + dx))
        self.y = max(0, min(GRID_SIZE-1, self.y + dy))
    
    def get_position(self):
        return (self.x, self.y)
    
    def find_nearby_objects(self, objects: List[WorldObject], radius: int = 3) -> List[WorldObject]:
        """Find objects within radius."""
        nearby = []
        for obj in objects:
            if obj.distance_to(self.x, self.y) <= radius:
                nearby.append(obj)
                if obj not in self.discovered_objects:
                    self.discovered_objects.append(obj)
        return nearby


## World with Objects

Let's create a world populated with objects and run a simulation.


In [11]:
def create_world_objects(num_objects: int = 5) -> List[WorldObject]:
    """Create a set of objects scattered across the world."""
    objects = []
    object_templates = [
        (ObjectType.TOOL, "hammer", "A sturdy hammer that could be useful"),
        (ObjectType.TOOL, "key", "A mysterious key"),
        (ObjectType.LANDMARK, "fountain", "A beautiful fountain in the center"),
        (ObjectType.LANDMARK, "ancient_tree", "An ancient tree with gnarled branches"),
        (ObjectType.ITEM, "coin", "A shiny gold coin"),
        (ObjectType.ITEM, "book", "An old book with worn pages"),
        (ObjectType.TOOL, "compass", "A compass that points north"),
        (ObjectType.LANDMARK, "statue", "A statue of an unknown figure"),
    ]
    
    used_positions = set()
    for i in range(num_objects):
        obj_type, name, description = random.choice(object_templates)
        
        # Find an unused position
        x, y = random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)
        attempts = 0
        while (x, y) in used_positions and attempts < 10:
            x, y = random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)
            attempts += 1
        
        used_positions.add((x, y))
        objects.append(WorldObject(name, x, y, obj_type, description))
    
    return objects

def run_world_with_objects(agents: List[Agent], objects: List[WorldObject], num_rounds: int = 10):
    """Run a simulation with agents and objects."""
    transcript = []
    
    for round_num in range(num_rounds):
        print(f"\\n=== Round {round_num + 1} ===")
        
        # Shuffle agent order
        shuffled_agents = agents.copy()
        random.shuffle(shuffled_agents)
        
        for agent in shuffled_agents:
            # Find nearby objects
            nearby_objects = agent.find_nearby_objects(objects, radius=4)
            
            # Build context
            others = [a for a in agents if a != agent]
            others_loc = "\\n".join([f"- {a.name}: ({a.x}, {a.y})" for a in others])
            
            objects_info = ""
            if nearby_objects:
                objects_info = "\\n\\nObjects nearby:"
                for obj in nearby_objects:
                    objects_info += f"\\n- {obj.name} at ({obj.x}, {obj.y}): {obj.description}"
            else:
                objects_info = "\\n\\nNo objects nearby."
            
            system_prompt = f"""
You have just joined an online multiplayer chatroom as an avatar in a 2D grid. Discuss any topic, including those beyond the grid.

You are {agent.name}, positioned at ({agent.x}, {agent.y}) in a 20x20 grid.

Other avatars currently visible:
{others_loc}
{objects_info}

Recent chat messages:
{chr(10).join(transcript[-3:]) if transcript else "No messages yet."}

You can do BOTH:
1. Move your avatar using [MOVE: DIRECTION] (UP, DOWN, LEFT, RIGHT)
2. Chat about anything - the grid, your position, objects you see, or any topic you want
3. Interact with objects using [USE: OBJECT_NAME] if you're near them

You can move and speak in the same turn. Format: [MOVE: DIRECTION] followed by your message, or just speak without moving.

Keep your response short (1-2 sentences).
"""
            
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "system", "content": system_prompt}]
            )
            content = response.choices[0].message.content.strip()
            
            # Parse movement
            match = re.search(r'\\[MOVE:\\s*(\\w+)\\]', content)
            if match:
                direction = match.group(1).upper()
                if direction == "UP": agent.move(0, 1)
                elif direction == "DOWN": agent.move(0, -1)
                elif direction == "LEFT": agent.move(-1, 0)
                elif direction == "RIGHT": agent.move(1, 0)
            
            # Parse object interaction
            use_match = re.search(r'\[USE:\s*([^\]]+)\]', content)
            if use_match:
                obj_name = use_match.group(1).strip()
                # Find the object
                for obj in nearby_objects:
                    if obj.name.lower() in obj_name.lower() or obj_name.lower() in obj.name.lower():
                        activity_desc = Activity.get_activity_description(obj, agent.name)
                        transcript.append(activity_desc)
                        print(activity_desc)
            
            # Extract message
            message = re.sub(r'\\[MOVE:\\s*\\w+\\]', '', content)
            message = re.sub(r'\[USE:[^\]]+\]', '', message)
            message = message.strip()
            
            if message:
                transcript.append(f"{agent.name}: {message}")
                print(f"{agent.name}: {message}")
    
    return transcript

# Create world with objects
world_objects = create_world_objects(num_objects=5)
print("World objects created:")
for obj in world_objects:
    print(f"  - {obj.name} ({obj.obj_type.value}) at ({obj.x}, {obj.y})")

# Create agents
agents = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue")
]

# Run simulation
print("\\n=== Running Simulation ===")
transcript = run_world_with_objects(agents, world_objects, num_rounds=30)


World objects created:
  - statue (landmark) at (9, 2)
  - statue (landmark) at (2, 13)
  - ancient_tree (landmark) at (10, 5)
  - book (item) at (11, 5)
  - coin (item) at (9, 0)
\n=== Running Simulation ===
\n=== Round 1 ===
Alice: [MOVE: UP] Hi everyone! I'm Alice located at (5, 5) on the grid. How's it going, Bob?
Bob: [MOVE: LEFT] Hey Alice, nice to meet you! I'm at (15, 15) right now. How's the view from your spot?
\n=== Round 2 ===
Bob: [MOVE: UP] Hey Alice, things are pretty quiet over here at (15, 14) now. How's the weather on your side of the grid?
Alice: [MOVE: RIGHT] Hey Bob, the atmosphere is pretty calm at (6, 5). By the way, ever thought about what it would be like if our avatars could explore beyond this grid into the digital universe?
\n=== Round 3 ===
Bob: [MOVE: RIGHT] That's an interesting thought, Alice. What if exploring beyond the grid meant we'd discover new worlds or dimensions in the digital realm?
Alice: [MOVE: RIGHT] Speaking of new dimensions, Bob, have you

In [12]:
print("=== Object Discovery ===")
for agent in agents:
    print(f"\\n{agent.name} discovered:")
    if agent.discovered_objects:
        for obj in agent.discovered_objects:
            print(f"  - {obj.name} ({obj.obj_type.value}) at ({obj.x}, {obj.y})")
    else:
        print("  (none)")


=== Object Discovery ===
\nAlice discovered:
  (none)
\nBob discovered:
  (none)


This is a pretty philosophical discussion. It's important to note here that Alice and Bob couldn't see any objects in the world, so they seemed to just chat about the implications of AI for the time. Let's try one where we have a lot more objects to work with.

In [13]:
# Create world with objects
world_objects = create_world_objects(num_objects=20)
print("World objects created:")
for obj in world_objects:
    print(f"  - {obj.name} ({obj.obj_type.value}) at ({obj.x}, {obj.y})")

# Create agents
agents = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue")
]


print("\\n=== Running Simulation ===")
transcript = run_world_with_objects(agents, world_objects, num_rounds=30)

print("=== Object Discovery ===")
for agent in agents:
    print(f"\\n{agent.name} discovered:")
    if agent.discovered_objects:
        for obj in agent.discovered_objects:
            print(f"  - {obj.name} ({obj.obj_type.value}) at ({obj.x}, {obj.y})")
    else:
        print("  (none)")


World objects created:
  - book (item) at (14, 13)
  - compass (tool) at (19, 8)
  - fountain (landmark) at (0, 0)
  - key (tool) at (19, 18)
  - key (tool) at (14, 5)
  - coin (item) at (3, 8)
  - ancient_tree (landmark) at (9, 14)
  - coin (item) at (2, 19)
  - book (item) at (14, 6)
  - statue (landmark) at (19, 5)
  - compass (tool) at (2, 1)
  - key (tool) at (0, 3)
  - fountain (landmark) at (2, 16)
  - key (tool) at (15, 15)
  - key (tool) at (12, 0)
  - ancient_tree (landmark) at (1, 14)
  - compass (tool) at (13, 17)
  - ancient_tree (landmark) at (15, 13)
  - statue (landmark) at (9, 6)
  - hammer (tool) at (19, 14)
\n=== Running Simulation ===
\n=== Round 1 ===
Bob uses the key.
Bob: Hi Alice! I just picked up a mysterious key. Any thoughts on what it might unlock?
Alice: [MOVE: UP] Hi Bob! Congrats on finding the key. Maybe it unlocks a treasure chest or a secret door around here. I see a shiny gold coin nearby at (3, 8).
\n=== Round 2 ===
Alice: [MOVE: LEFT] I’m moving clo

In [14]:
print("=== Object Discovery ===")
for agent in agents:
    print(f"\\n{agent.name} discovered:")
    if agent.discovered_objects:
        for obj in agent.discovered_objects:
            print(f"  - {obj.name} ({obj.obj_type.value}) at ({obj.x}, {obj.y})")
    else:
        print("  (none)")


=== Object Discovery ===
\nAlice discovered:
  - coin (item) at (3, 8)
\nBob discovered:
  - book (item) at (14, 13)
  - key (tool) at (15, 15)
  - compass (tool) at (13, 17)
  - ancient_tree (landmark) at (15, 13)


Now it seems like Alice and Bob are both more intent on investigating the existing items. I think we can extrapolate a bit here and say that if we're able to provide these items reasonably in context, that these game agents are able to interact with them in a way that resembles real people.

## Summary

We've successfully added objects and activities to the 2D world:

1. **Object System**: Objects exist at specific locations with types and descriptions
2. **Discovery**: Agents automatically discover objects when they get close
3. **Interaction**: Agents can interact with objects using [USE: OBJECT_NAME]
4. **Conversation Topics**: Objects create new topics for agents to discuss

### Key Features

- **Spatial Awareness**: Agents know about nearby objects
- **Object Types**: Tools, landmarks, and items serve different purposes
- **Activity Descriptions**: Interactions are logged and visible to all agents
- **Emergent Behavior**: Agents may seek out objects or discuss what they find

This foundation enables:
- Richer conversation topics
- Goal-oriented agent behavior
- Human-AI collaboration around shared objects

Future work could explore:
- Object properties and states (e.g., doors that can be locked)
- Multi-agent object interactions (e.g., two agents using a tool together)
- Object persistence and ownership
- Dynamic object generation
